In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import json
import os
import re

In [87]:
def removeTags(segments):
    ortho = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", segments)
    ortho = ortho.replace("[]","-")[:-1]

    return ortho

In [94]:
datasets = ['zulu', 'xhosa', 'ndebele', 'swati']
datasetTypes = ['train', 'dev', 'test']
data = open('../Data/swati/swati.clean.test.conll').read()

In [95]:
dataDict = {}
dataDict['data'] = []
dataArr = []

In [96]:
file = open("../Data/swati/json/swati-test.txt", "w")

for line in data.split('\n'):
    line = line.split(' | ')
    x = {
      "src": line[0],
      "trg": removeTags(line[3]),

    }
    
    y = json.dumps(x)
    file.write(str(y)+'\n')
file.close()
print("Done Writing")

IndexError: list index out of range

In [2]:
def tokenize(text):
    
    return [char for char in text]
    


In [3]:
SRC = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)
TRG = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

In [4]:
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

In [5]:
train_data, test_data, valid_data = torchtext.data.TabularDataset.splits(
                            path = "",
                            train = 'train.json',
                            test = 'test.json',
                            validation = 'validation.json',
                            format = 'json',
                            fields = fields
)
#train_data = train_data[0]

In [6]:

print(vars(train_data[0]))
print(len(train_data))
print(len(test_data))
print(len(valid_data))

{'src': ['n', 'g', 'e', 'z', 'i', 'n', 'k', 'o', 'n', 'z', 'o'], 'trg': ['n', 'g', 'a', '[', 'n', 'p', 'r', 'e', ']', 'i', '[', 'n', 'p', 'r', 'e', 'p', 'r', 'e', ']', 'z', 'i', 'n', '[', 'b', 'p', 'r', 'e', ']', 'k', 'o', 'n', 'z', 'o', '[', 'n', 's', 't', 'e', 'm', ']']}
17778
1777
3298


In [228]:
SRC.build_vocab(train_data)
TRG.build_vocab(train_data)

In [229]:
print(SRC.vocab.itos)
print(TRG.vocab.itos)

['<unk>', '<pad>', '<sos>', '<eos>', 'a', 'e', 'i', 'n', 'o', 'u', 'l', 'k', 'h', 's', 'z', 'm', 'w', 'b', 'g', 't', 'y', 'd', 'p', 'f', 'q', 'c', 'v', 'j', 'r', 'x', '0', '1', '%', '3', '2', '5', '8', ',', '4', '9', '6', '7', '.', '"', "'"]
['<unk>', '<pad>', '<sos>', '<eos>', 'e', '[', ']', 'r', 'p', 'n', 'o', 't', 's', 'a', 'm', 'b', 'c', 'i', 'v', 'u', 'l', '1', 'k', '5', 'h', 'd', 'z', 'x', 'f', 'g', 'w', '9', 'y', '7', '3', '2', '0', '6', '4', 'j', '8', 'q', ',', ' ', '(', ')', '%', '.', '"', "'"]


In [230]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: x.s, #sort by s attribute (quote)
    batch_size=BATCH_SIZE,
    device=device)

In [232]:
print('Train:')

loss = 0
for batch in train_iterator:
    src = batch.src
    trg = batch.trg 
    print(src)
    
    

Train:
tensor([[ 2, 13,  6, 19, 12,  9, 19, 12,  9, 11,  6, 13,  5,  3,  1,  1,  1,  1],
        [ 2, 20,  8, 17,  9, 22, 12,  8, 20,  6, 13,  4,  3,  1,  1,  1,  1,  1],
        [ 2,  8,  7,  5, 18,  9,  7, 20,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 11,  9, 21,  5, 21,  5, 10, 16,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 20,  5, 14,  6, 10, 16,  4,  7,  5,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  9, 11,  9, 11, 12,  8, 11, 12,  5, 10, 16,  4,  3,  1,  1,  1,  1],
        [ 2,  4, 11,  4, 11,  4, 24,  4, 10,  6,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  7,  4, 10,  5, 10,  8,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 13,  6, 11, 12,  9, 10,  9, 15,  5,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  7, 18,  8, 11, 16,  5, 14,  6,  7, 11,  4, 15, 17,  6, 13,  8,  3],
        [ 2,  5, 11, 12,  8, 11, 12, 16,  4, 20,  8,  3,  1,  1,  1,  1,  1,  1],
        [ 2,  5, 13,  4, 14,  6, 13, 16,  5,  7,  6,  3,  1,  1,  1,  1,  1,  1],
        [

tensor([[ 2, 10, 16,  4, 13,  5, 11, 12,  4, 20,  4,  3,  1,  1,  1,  1,  1,  1,
          1,  1],
        [ 2,  7, 27,  5,  7, 18,  8, 11,  9, 19, 12,  5,  7, 18,  6, 13,  5, 10,
          4,  3],
        [ 2,  9, 11,  9, 11, 16,  4, 14,  6,  3,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1],
        [ 2,  4, 15,  4, 19, 13, 12,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1],
        [ 2, 16,  5, 14,  6,  7, 12, 10,  4,  7, 14,  6,  3,  1,  1,  1,  1,  1,
          1,  1],
        [ 2,  5, 14,  6, 15, 17,  4, 10, 16,  4,  3,  1,  1,  1,  1,  1,  1,  1,
          1,  1],
        [ 2,  6,  7,  8, 19, 12,  6, 13,  6,  3,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1],
        [ 2,  7,  8, 11,  9, 29, 12,  9, 15,  4,  7,  4,  3,  1,  1,  1,  1,  1,
          1,  1],
        [ 2,  8, 11,  9, 17, 12,  4, 10, 16,  5,  3,  1,  1,  1,  1,  1,  1,  1,
          1,  1],
        [ 2, 13,  6, 12,  4, 15, 17,  6, 13,  4,  7,  5,  3,  1,  1,  1,  1,  1,
          1,  1],
        [ 

tensor([[ 2,  5, 17,  5, 11, 16,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 14,  6, 10,  4,  7, 21,  5, 10, 16,  5,  3,  1,  1,  1,  1,  1],
        [ 2,  4, 19, 12,  8, 10,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 10,  8, 11,  9, 21, 10,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  6, 15,  6, 14, 16,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 11,  9, 17,  5, 11, 16,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 13,  8, 11,  9, 24,  4, 10,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  4, 13, 12,  5, 13, 12,  4, 20,  8,  3,  1,  1,  1,  1,  1,  1],
        [ 2, 13,  8, 11,  9, 12,  4, 15, 17,  4,  3,  1,  1,  1,  1,  1,  1],
        [ 2,  9, 16,  5, 21, 16,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  5, 15, 17,  6,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 10,  8, 11,  9, 12,  9, 10,  9, 15,  5,  7,  6,  3,  1,  1,  1],
        [ 2, 11,  8, 11,  9, 22, 12,  5, 10,  4,  3,  1,  1,  1,

tensor([[ 2, 20,  8, 11,  9, 19, 12,  9, 15,  5, 10,  4,  3,  1,  1,  1,  1],
        [ 2,  7,  5, 13,  6, 25,  5, 10,  8,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 10,  5, 14,  8, 10,  6, 15,  8,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  5, 13,  6, 25, 12,  4, 14,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  5, 13,  6,  7, 18,  4, 22, 12,  4,  7, 21, 10,  5,  3,  1,  1],
        [ 2,  7, 18,  8, 11,  9, 10,  6,  7, 18,  4,  7,  4, 20,  8,  3,  1],
        [ 2, 13,  8, 15,  7,  6, 11,  4, 14,  6,  3,  1,  1,  1,  1,  1,  1],
        [ 2, 13,  5, 15, 22,  6, 10,  8,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 10, 16,  5, 14,  6, 23,  9,  7, 21,  8,  3,  1,  1,  1,  1,  1],
        [ 2, 11,  9,  7, 18,  4, 19, 12,  4, 19, 12,  4,  3,  1,  1,  1,  1],
        [ 2,  9, 11, 12,  9, 10,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  8, 11,  9, 24, 12,  9, 17,  5, 11,  4, 20,  8,  3,  1,  1,  1],
        [ 2,  7,  8, 11,  9, 14,  6, 11, 12,  9, 10,  6, 13,  4,

tensor([[ 2, 10,  6, 20,  6, 10,  9,  7, 18,  9,  3,  1,  1,  1,  1,  1],
        [ 2, 11,  9,  7, 18,  6, 19, 12,  8, 11,  8, 14,  6, 13,  4,  3],
        [ 2,  5, 11, 12,  6, 13, 12, 16,  4,  3,  1,  1,  1,  1,  1,  1],
        [ 2,  7,  8, 11,  9, 10,  8,  7, 21,  8, 10,  8, 14,  4,  3,  1],
        [ 2,  5, 18,  9,  7, 20,  4, 14, 16,  5,  3,  1,  1,  1,  1,  1],
        [ 2,  9, 11,  9,  7, 25,  6,  7, 19,  6, 13,  4,  7,  4,  3,  1],
        [ 2,  8, 14,  6, 15,  6, 13,  5, 10,  5,  3,  1,  1,  1,  1,  1],
        [ 2,  9, 11,  9, 13,  9, 11,  5, 10,  4,  3,  1,  1,  1,  1,  1],
        [ 2,  5, 14,  6, 19, 13, 12,  4, 10,  6, 16,  5,  3,  1,  1,  1],
        [ 2,  6, 15,  6, 19, 12,  5, 13, 12, 16,  4,  7,  4,  3,  1,  1],
        [ 2, 14,  8, 11,  9, 22, 12,  4, 19, 12,  4,  3,  1,  1,  1,  1],
        [ 2,  8, 11,  9, 14,  8, 11,  9, 19, 12,  6,  3,  1,  1,  1,  1],
        [ 2,  9, 15, 19, 12,  5, 19, 12,  8,  3,  1,  1,  1,  1,  1,  1],
        [ 2, 11, 16,  4, 15,  4, 27,  

tensor([[ 2, 14,  5,  7, 12, 10,  4, 10,  8,  3,  1,  1,  1,  1,  1,  1],
        [ 2, 13,  6,  7,  5, 14,  6, 15, 22,  5,  7, 21,  9, 10,  8,  3],
        [ 2, 11, 16,  5, 14,  6, 10, 16,  4,  7,  5,  3,  1,  1,  1,  1],
        [ 2, 10,  4, 17,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 14,  6, 12, 10,  4,  7, 18,  4,  7,  6, 13, 16,  5,  3,  1],
        [ 2,  6, 14,  6, 18,  9, 10,  6,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  5, 12, 10,  9, 22, 12,  5, 11,  6, 10,  5,  3,  1,  1,  1],
        [ 2,  9, 21,  8, 11,  8, 19,  5, 10,  4,  3,  1,  1,  1,  1,  1],
        [ 2,  5, 13,  6, 17,  9, 20,  5, 11,  5, 14,  6, 16,  5,  3,  1],
        [ 2,  5,  7,  5, 13,  6, 13,  6,  7, 21,  8,  3,  1,  1,  1,  1],
        [ 2,  9, 17,  9,  7, 18,  4, 11,  4,  7,  4,  7,  6,  3,  1,  1],
        [ 2,  7,  8, 11,  9, 24,  5, 21,  4,  3,  1,  1,  1,  1,  1,  1],
        [ 2,  6, 16,  5, 17, 13,  6, 19,  5,  3,  1,  1,  1,  1,  1,  1],
        [ 2,  4,  7, 18,  4, 16,  8,  

tensor([[ 2,  9, 14,  8, 19, 12,  8, 17,  5, 10,  4,  3,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2,  7, 27,  4, 10,  8,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2,  7,  8, 15,  9,  7, 19,  9,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2,  4, 10,  9,  7,  6, 11,  5, 14,  5,  3,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2,  7, 18,  4, 22, 12,  4, 15, 17,  6,  3,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2,  8, 11,  9,  7,  8, 11,  9, 14,  6, 24, 12,  5,  7, 20,  4,  3,  1,
          1,  1,  1],
        [ 2, 20,  5, 24, 12,  4, 16,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2, 11, 16,  5, 14,  6,  7, 18,  9, 24,  9, 11,  8,  3,  1,  1,  1,  1,
          1,  1,  1],
        [ 2, 20,  8, 11,  9, 24,  4, 10,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1],
        [ 2,  6, 14,  6, 19, 13, 12,  4, 10,  8,  3,  1,  1,  1,  1,  1, 

tensor([[ 2,  7, 18,  4, 17,  4,  7, 19,  9,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  4, 12, 10,  8, 13,  6, 16,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  9, 19, 12,  5,  7, 18,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  8, 21,  8, 11,  8, 19,  5, 10,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  4, 17,  4, 18,  4,  7,  4,  7,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 17,  4, 19, 12,  8, 10,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  7,  5, 14,  6,  7, 12, 10,  4,  7, 14,  6,  3,  1,  1,  1,  1,  1],
        [ 2,  5, 14,  5,  7, 14,  5, 11,  4,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 11,  9, 18, 25,  6,  7,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  4, 17,  4, 20,  6,  7, 11,  9, 10,  9,  7, 18, 16,  4,  7,  5,  3],
        [ 2,  7,  8, 15, 22, 12,  5,  7, 20,  6, 13,  6, 13,  6,  3,  1,  1,  1],
        [ 2,  4, 17,  4, 10,  9, 19, 12,  8, 10,  4, 20,  8,  3,  1,  1,  1,  1],
        [ 2,  8,

tensor([[ 2, 10,  5, 13,  6, 23,  9,  7, 21,  4, 14, 16,  5,  3,  1,  1,  1],
        [ 2,  5, 11,  9, 24,  6,  7,  6, 13,  5, 11,  6, 13,  5,  7,  6,  3],
        [ 2,  9, 11,  9, 23,  9,  7, 21,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 10,  5, 19, 15, 31,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  7,  4, 15,  4, 18,  4, 15,  4,  3,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  4, 11, 12,  8, 11, 12,  5, 10, 16,  4,  3,  1,  1,  1,  1,  1],
        [ 2, 11, 16,  5, 13,  6, 17,  8, 22, 12,  8,  3,  1,  1,  1,  1,  1],
        [ 2, 10, 16,  5, 14,  6, 15,  4, 10,  6,  3,  1,  1,  1,  1,  1,  1],
        [ 2, 11, 16,  4, 11,  9, 20,  6, 13,  6, 11, 12,  4, 19, 12,  6,  3],
        [ 2, 16,  4, 15,  4, 10,  9,  7, 18,  5, 10,  8,  3,  1,  1,  1,  1],
        [ 2,  5, 11,  9, 12, 10,  8, 10,  5,  7,  6,  3,  1,  1,  1,  1,  1],
        [ 2,  7,  8, 11,  9,  7, 20,  5,  3,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 11, 16,  5,  7, 19,  4,  7, 21,  8,  3,  1,  1,  1,

KeyboardInterrupt: 